# COMMON DEFINE

In [ ]:
from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.core import Lambda, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from keras.regularizers import l2
K.set_image_data_format('channels_last')
import csv
import os
import numpy as np
import pandas as pd

In [ ]:
def load_data(i):
    data=pd.read_csv('data/siamese_data_merge/'+str(i)+'.csv', 'r', delimiter=',')
    train, val=train_test_split(data, test_size=0.1)
    
    X_train1=[]
    X_train2=[]
    Y_train=[]
    train_length=len(train)
    for j in range(0, train_length):
        X_train1.append(train.iloc[j][0:38].values)
        X_train2.append(train.iloc[j][38:76].values)
        Y_train.append(train.iloc[j][76])
        
    X_val1=[]
    X_val2=[]
    Y_val=[]
    val_length=len(val)
    for j in range(0, val_length):
        X_val1.append(val.iloc[j][0:38].values)
        X_val2.append(val.iloc[j][38:76].values)
        Y_val.append(val.iloc[j][76])

    X_train1=np.reshape(X_train1, [train_length, 38])
    X_train2=np.reshape(X_train2, [train_length, 38])
    Y_train=np.reshape(Y_train, [train_length, 1])
    X_val1=np.reshape(X_val1, [val_length, 38])
    X_val2=np.reshape(X_val2, [val_length, 38])
    Y_val=np.reshape(Y_val, [val_length, 1])
    
    return [X_train1, X_train2, Y_train, X_val1, X_val2, Y_val]

In [ ]:
def initialize_weights(shape, dtype=np.float32):
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)
def initialize_bias(shape, dtype=np.float32):
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [ ]:
def get_siamese_model():
    left_input = Input(shape=(38,))
    right_input = Input(shape=(38,))
    model = Sequential()
    model.add(Dense(64, activation='relu',
                    input_shape=(38,),
                    kernel_regularizer=l2(1e-3),
                    kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    model.add(Dense(512, activation='relu',
                    kernel_regularizer=l2(1e-3),
                    kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    model.add(Dense(512, activation='relu',
                    kernel_regularizer=l2(1e-3),
                    kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid', bias_initializer=initialize_bias)(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    return siamese_net

In [ ]:
model=get_siamese_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')
for i in range(0, 16):
    z=load_data(i)
    hist=model.fit([z[0], z[1]], z[2], validation_data=([z[3], z[4]], z[5]), batch_size=128, epochs=10, callbacks=[early_stopping])
    model.save('model/test'+str(i)+'.h5')